## **1. Import Libraries**

In [14]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model # To build a neural network model layer by layer , To load a saved model for predictions or further training.
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # To add layers to the neural network:
from tensorflow.keras.utils import to_categorical


## **2. Prepare Dataset**

In [15]:
def load_data(base_dir):
    data = []
    labels = []
    label_dict = {i: name for i, name in enumerate(sorted(os.listdir(base_dir))) if os.path.isdir(os.path.join(base_dir, name))}

    for label, person_name in label_dict.items():
        person_dir = os.path.join(base_dir, person_name)
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (100, 100))  # Resize to 100x100
            data.append(img)
            labels.append(label)

    data = np.array(data).reshape(-1, 100, 100, 1)  # Add channel dimension
    labels = np.array(labels)
    return data, labels, label_dict

# Load Dataset
base_dir = "./Dataset"  # Path to the dataset
data, labels, label_dict = load_data(base_dir)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Normalize Data
X_train = X_train / 255.0
X_test = X_test / 255.0

# One-Hot Encode Labels
y_train = to_categorical(y_train, num_classes=len(label_dict))
y_test = to_categorical(y_test, num_classes=len(label_dict))



---

### **Code Explanation with "Why?"**

#### **`def load_data(base_dir):`**
- **Why**: This function organizes the process of loading, processing, and labeling images from a dataset folder.

---

#### **`data = []` and `labels = []`**
- **Why**: `data` will store the processed image data, and `labels` will store corresponding numeric labels to match images with their categories.

---

#### **`label_dict = {i: name for i, name in enumerate(sorted(os.listdir(base_dir))) if os.path.isdir(os.path.join(base_dir, name))}`**
- **Why**: 
  - Creates a mapping (dictionary) between numbers (`0, 1, 2, ...`) and category names (folder names).
  - Helps assign a numeric label to each category so models can process them.

---

#### **`for label, person_name in label_dict.items():`**
- **Why**: Iterates over each folder (category) to access its images.

---

#### **`person_dir = os.path.join(base_dir, person_name)`**
- **Why**: Combines the base directory path with the folder name to get the full path to the current category's folder.

---

#### **`for image_name in os.listdir(person_dir):`**
- **Why**: Loops through each file (image) in the current category's folder.

---

#### **`image_path = os.path.join(person_dir, image_name)`**
- **Why**: Constructs the full file path to the image.

---

#### **`img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)`**
- **Why**: Reads the image in grayscale format.
  - Grayscale simplifies the input (less computational load) by reducing it to one color channel.

---

#### **`img = cv2.resize(img, (100, 100))`**
- **Why**: Resizes the image to `100x100` pixels.
  - Ensures all images have the same size, which is required for feeding them into the model.

---

#### **`data.append(img)` and `labels.append(label)`**
- **Why**:
  - Adds the processed image (`img`) to the `data` list.
  - Adds the numeric category label to the `labels` list.

---

#### **`data = np.array(data).reshape(-1, 100, 100, 1)`**
- **Why**: Converts `data` into a NumPy array and reshapes it.
  - The `-1` ensures the array automatically adjusts its size.
  - `(100, 100, 1)` includes a channel dimension for grayscale images.

---

#### **`labels = np.array(labels)`**
- **Why**: Converts the list of labels into a NumPy array for easier manipulation.

---

#### **`return data, labels, label_dict`**
- **Why**: Returns the processed data, labels, and the dictionary for later use.

---

### **Loading Dataset**
```python
base_dir = "./Dataset"
data, labels, label_dict = load_data(base_dir)
```
- **Why**: Calls the `load_data` function to load all images and labels from the folder `./Dataset`.

---

### **Splitting Data**
```python
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
```
- **Why**: Divides the dataset into training (80%) and testing (20%) sets.
  - Training data is used to teach the model.
  - Testing data evaluates how well the model learned.

---

### **Normalizing Data**
```python
X_train = X_train / 255.0
X_test = X_test / 255.0
```
- **Why**: Scales pixel values (which range from `0` to `255`) to values between `0` and `1`.
  - Normalized data helps the model train faster and perform better.

---

### **One-Hot Encoding Labels**
```python
y_train = to_categorical(y_train, num_classes=len(label_dict))
y_test = to_categorical(y_test, num_classes=len(label_dict))
```
- **Why**: Converts numeric labels into a one-hot encoded format (e.g., `2 → [0, 0, 1]`).
  - This format is better for training classification models, especially in TensorFlow.

---


**OVERALL:**
This code prepares image data for training a machine learning model. It starts by loading images from different category folders, assigning each folder a numeric label, and processing the images by converting them to grayscale, resizing them to a uniform size (100x100), and storing them in arrays. The data is split into training and testing sets to evaluate the model's performance. Pixel values are normalized to a range of 0 to 1 for better model learning, and labels are converted into one-hot encoded format for classification. Overall, it organizes the raw dataset into a format that a neural network can efficiently process for image recognition tasks.

## **3. Train the CNN Model**

In [16]:
# Define CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_dict), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Save the Model
model.save('face_recognition_cnn.h5')


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 364ms/step - accuracy: 0.5562 - loss: 0.7936 - val_accuracy: 0.4750 - val_loss: 0.6686
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 266ms/step - accuracy: 0.5859 - loss: 0.6637 - val_accuracy: 0.5250 - val_loss: 0.6746
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 268ms/step - accuracy: 0.6801 - loss: 0.6564 - val_accuracy: 0.9500 - val_loss: 0.6504
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - accuracy: 0.7293 - loss: 0.6123 - val_accuracy: 1.0000 - val_loss: 0.5302
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 238ms/step - accuracy: 0.8441 - loss: 0.4499 - val_accuracy: 0.8750 - val_loss: 0.3187
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step - accuracy: 0.8471 - loss: 0.3592 - val_accuracy: 1.0000 - val_loss: 0.2109
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step - accuracy: 0.9174 - loss: 0.2623 - val_accuracy: 1.0000 - val_loss: 0.1013
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 284ms/step - accuracy: 0.9670 - loss: 0.1476 - val_accuracy: 1.0000 - val_loss:

---

### **Define CNN Model**
```python
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_dict), activation='softmax')
])
```
- **Why**: This builds a Convolutional Neural Network (CNN) layer by layer to process and classify the image data.
  - **`Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1))`**: 
    - Adds a convolutional layer with 32 filters and a kernel size of `(3, 3)`.
    - Extracts image features like edges and textures.
    - `input_shape=(100, 100, 1)` specifies the input image dimensions.
    - `relu` activation removes negative values, keeping only important features.
  - **`MaxPooling2D((2, 2))`**: Reduces the size of feature maps to make the model faster and prevent overfitting.
  - **`Dropout(0.2)`**: Randomly disables 20% of neurons during training to prevent overfitting.
  - **`Conv2D(64, (3, 3), activation='relu')`**: Adds another convolutional layer with 64 filters for deeper feature extraction.
  - **`Flatten()`**: Converts the 2D feature maps into a 1D vector, preparing it for dense (fully connected) layers.
  - **`Dense(128, activation='relu')`**: Fully connected layer with 128 neurons for learning complex patterns.
  - **`Dropout(0.5)`**: Disables 50% of neurons to avoid overfitting further.
  - **`Dense(len(label_dict), activation='softmax')`**: Output layer with as many neurons as categories in `label_dict`.
    - `softmax` converts predictions into probabilities for multi-class classification.

---

### **Compile the Model**
```python
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
```
- **Why**: Configures the model for training.
  - **`optimizer='adam'`**: Optimizes weights efficiently during training.
  - **`loss='categorical_crossentropy'`**: Calculates loss for multi-class classification.
  - **`metrics=['accuracy']`**: Tracks accuracy during training.

---

### **Train the Model**
```python
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
```
- **Why**: Fits the model to the training data and evaluates it using the validation data.
  - **`epochs=10`**: Trains the model for 10 complete passes through the dataset.
  - **`batch_size=32`**: Processes 32 samples at a time for efficient training.
  - **`validation_data=(X_test, y_test)`**: Uses the test set to monitor accuracy and loss after each epoch.

---

### **Save the Model**
```python
model.save('face_recognition_cnn.h5')
```
- **Why**: Saves the trained model to a file so it can be reused without retraining.

---

### **Overall Summary**
This code defines a CNN for image classification, compiles it for training, trains it on processed image data, and saves the trained model for future use. The CNN extracts features from images using convolution and pooling layers, learns patterns with dense layers, and outputs probabilities for each class.

## **4. Real-Time Face Recognition**

In [17]:
# Load Haar Cascade for face detection
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load the trained CNN model
model = load_model('face_recognition_cnn.h5')

# Load label dictionary
label_dict = {i: name for i, name in enumerate(sorted(os.listdir(base_dir))) if os.path.isdir(os.path.join(base_dir, name))}



---

### **Load Haar Cascade for Face Detection**
```python
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
```
- **Why**: This loads a pre-trained Haar Cascade classifier for detecting faces in images or video frames.
  - `haarcascade_frontalface_default.xml` is a file containing data to identify features of human faces.
  - This classifier is efficient for quickly detecting face regions in real-time.

---

### **Load the Trained CNN Model**
```python
model = load_model('face_recognition_cnn.h5')
```
- **Why**: Loads the previously trained CNN model from the saved file (`face_recognition_cnn.h5`).
  - This model will be used to predict the identity of the detected face based on the trained categories.

---

### **Load Label Dictionary**
```python
label_dict = {i: name for i, name in enumerate(sorted(os.listdir(base_dir))) if os.path.isdir(os.path.join(base_dir, name))}
```
- **Why**: Recreates the `label_dict` mapping numeric labels (`0, 1, 2, ...`) to their corresponding category names (e.g., `0: "Person A"`).
  - Ensures consistency between the categories used during training and the labels during prediction.
  - Example: If `base_dir` contains folders `["John", "Jane"]`, the dictionary might look like `{0: 'Jane', 1: 'John'}`.

---

### **Overall Summary**
This code sets up face detection using a Haar Cascade, loads the pre-trained CNN model for recognizing faces, and rebuilds the label dictionary to map numeric labels to category names. These components are essential for performing face recognition in real-time or on static images.

In [ ]:
# Initialize Webcam
cap = cv2.VideoCapture(1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Preprocess the detected face
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (100, 100))
        face = face.reshape(1, 100, 100, 1) / 255.0

        # Predict the person's name
        prediction = model.predict(face)
        label_index = np.argmax(prediction)
        confidence = np.max(prediction)
        person_name = label_dict[label_index] if confidence > 0.7 else "Unknownnn"

        # Draw a rectangle and put the label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, person_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    cv2.imshow("Face Recognition", frame)
    if cv2.waitKey(1) == 13:  # Press Enter to exit
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/

Here’s the detailed explanation for this code, focusing on why each line or section is used:

---

### **Initialize Webcam**
```python
cap = cv2.VideoCapture(1)
```
- **Why**: Captures live video from the webcam.
  - `1` selects the second connected camera (index `0` is usually the default webcam).
  - You can change this number based on the connected camera.

---

### **Start the Infinite Loop**
```python
while True:
```
- **Why**: Continuously processes each frame from the webcam until manually stopped.

---

### **Capture a Frame**
```python
ret, frame = cap.read()
if not ret:
    break
```
- **Why**:
  - `cap.read()` captures a single frame from the webcam.
  - `ret` is `True` if a frame was successfully captured, otherwise `False`.
  - If capturing fails, the loop exits to avoid errors.

---

### **Convert to Grayscale**
```python
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
```
- **Why**: Converts the captured frame from color (BGR) to grayscale.
  - Face detection and recognition typically work faster and just as accurately on grayscale images.

---

### **Detect Faces**
```python
faces = face_classifier.detectMultiScale(gray, 1.3, 5)
```
- **Why**:
  - Detects faces in the grayscale image using the Haar Cascade classifier.
  - `1.3`: Scale factor for reducing the image size during detection.
  - `5`: Minimum neighbors required for a region to be considered a face.
  - Returns a list of coordinates (`x, y, w, h`) for detected faces.

---

### **Process Each Detected Face**
```python
for (x, y, w, h) in faces:
```
- **Why**: Loops through all detected faces and processes them one by one.

---

### **Crop and Resize the Face**
```python
face = gray[y:y+h, x:x+w]
face = cv2.resize(face, (100, 100))
face = face.reshape(1, 100, 100, 1) / 255.0
```
- **Why**:
  - Extracts the face region from the grayscale image using coordinates.
  - Resizes it to `100x100`, matching the input size expected by the CNN model.
  - Reshapes it into the required 4D shape `(1, 100, 100, 1)` for the model.
  - Normalizes pixel values to a range of `0-1` for better predictions.

---

### **Predict the Person**
```python
prediction = model.predict(face)
label_index = np.argmax(prediction)
confidence = np.max(prediction)
person_name = label_dict[label_index] if confidence > 0.7 else "Unknown"
```
- **Why**:
  - `model.predict(face)`: Predicts probabilities for each category (person).
  - `np.argmax(prediction)`: Finds the label with the highest probability.
  - `np.max(prediction)`: Finds the confidence (highest probability value).
  - If confidence exceeds `0.7` (threshold), the predicted name from `label_dict` is displayed; otherwise, it shows "Unknown."

---

### **Draw Rectangle and Label**
```python
cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
cv2.putText(frame, person_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
```
- **Why**:
  - Draws a rectangle around the detected face in the live video frame.
  - Displays the predicted name (or "Unknown") above the rectangle.

---

### **Show the Processed Frame**
```python
cv2.imshow("Face Recognition", frame)
if cv2.waitKey(1) == 13:  # Press Enter to exit
    break
```
- **Why**:
  - Displays the live video feed with annotations (face rectangles and names).
  - Waits for the `Enter` key (ASCII code `13`) to exit the loop.

---

### **Release Resources and Close Windows**
```python
cap.release()
cv2.destroyAllWindows()
```
- **Why**:
  - Releases the webcam and associated resources to avoid memory or hardware issues.
  - Closes all OpenCV windows displaying the video.

---

### **Overall Summary**
This code uses a webcam to capture live video, detects faces in each frame using the Haar Cascade, and identifies the detected faces using a pre-trained CNN model. It displays the video with rectangles and labels drawn around recognized faces in real time. Unrecognized faces are labeled as "Unknown," and the program exits when the `Enter` key is pressed.